# Data Cleaning for URP 353, Assignment 2

In [1]:
import pandas as pd

## Read cached yelp data and drop NAs

In [2]:
food_diversity = pd.read_json('food_diversity.json', orient = 'records')

In [3]:
food_diversity.shape

(3876, 5)

In [4]:
food_diversity.sample(5)

,query,category,total_business_count,sample_rating,sample_review_count
3702,"Wichita, Kansas",persian,0,NA,NA
2210,"Las Vegas, Nevada",austrian,0,NA,NA
171,"Chicago, Illinois",cuban,31,4.20968,315.452
3294,"Miami, Florida",german,5,3.8,197.2
305,"Philadelphia, Pennsylvania",african,23,3.73913,25.3043


In [5]:
food_diversity.head(5)

,query,category,total_business_count,sample_rating,sample_review_count
0,"New York, New York",afghani,18,3.75,150.111
1,"New York, New York",african,48,4.11458,115.646
2,"New York, New York",arabian,5,3.9,51.6
3,"New York, New York",argentine,21,3.90476,182.857
4,"New York, New York",armenian,5,4.5,27.4


Look at summary statistics for total_business count

In [6]:
food_diversity.describe()

,total_business_count
count,3876.000000
mean,25.761868
std,94.006502
min,0.000000
25%,0.000000
50%,3.000000
75%,10.000000
max,1800.000000


Look at number of NAs and Errors

In [7]:
food_diversity['total_business_count'].value_counts()

0       1204
1        422
5        262
2        256
6        172
        ... 
1800       1
545        1
1600       1
1400       1
939        1
Name: total_business_count, Length: 274, dtype: int64

In [8]:
food_diversity['sample_rating'].value_counts()

NA                   1187
4.0                   329
4.5                   265
4.25                  131
3.75                   82
                     ... 
4.235294117647059       1
3.191176470588235       1
3.1                     1
4.105263157894737       1
3.802325581395349       1
Name: sample_rating, Length: 409, dtype: int64

In [9]:
food_diversity['sample_rating'].value_counts().loc['Error']

17

### Calculate Server Stataus 500 error rate

In [10]:
food_diversity['sample_rating'].value_counts().loc['Error'] / food_diversity.shape[0]

0.0043859649122807015

### Drop categories with 0 businesses and errors

In [11]:
food_diversity = food_diversity[ (food_diversity['sample_rating'] != 'Error') 
                                & (food_diversity['sample_rating'] != 'NA')]
food_diversity.shape

(2672, 5)

## Create country data to look up each food category for mapping

External data set from github user Dinuks, ISO 3166-1 Country + Nationality listing in multiple formats

In [12]:
cetegories = pd.DataFrame(food_diversity['category'].unique())
cetegories['category'] = cetegories[0].str.capitalize()
cetegories

,0,category
0,afghani,Afghani
1,african,African
2,arabian,Arabian
3,argentine,Argentine
4,armenian,Armenian
...,...,...
71,uzbek,Uzbek
72,vietnamese,Vietnamese
73,nicaraguan,Nicaraguan
74,bulgarian,Bulgarian


In [13]:
country_codes = pd.read_csv('https://raw.githubusercontent.com/Dinuks/country-nationality-list/master/countries.csv')

In [14]:
country_codes.sample(1)

,num_code,alpha_2_code,alpha_3_code,en_short_name,nationality
31,76,BR,BRA,Brazil,Brazilian


In [15]:
country_lookup = pd.merge( country_codes, cetegories, how = 'inner', left_on = 'nationality', right_on = 'category' )

In [16]:
country_lookup = country_lookup[['num_code', 'en_short_name', 'category' ]]

In [17]:
country_lookup.shape

(40, 3)

Manually add non-matching countries

In [18]:
[x for x in cetegories['category'].to_list() if x not in country_codes['nationality'].to_list()]

['Afghani',
 'African',
 'Arabian',
 'Basque',
 'British',
 'Cajun',
 'Caribbean',
 'Catalan',
 'Filipino',
 'Halal',
 'Hawaiian',
 'Himalayan',
 'Hungarian',
 'Iberian',
 'Korean',
 'Kosher',
 'Laotian',
 'Latin',
 'Mediterranean',
 'Mideastern',
 'Modern_european',
 'Newamerican',
 'Newmexican',
 'Panasian',
 'Persian',
 'Piadina',
 'Polynesian',
 'Scandinavian',
 'Scottish',
 'Slovakian',
 'Somali',
 'Srilankan',
 'Taiwanese',
 'Tradamerican',
 'Uzbek',
 'Guamanian']

In [19]:
country_lookup = country_lookup.append({'num_code': 826,
                       'en_short_name':'United Kingdom of Great Britain and Northern Ireland',
                       'category': 'British'}, ignore_index=True).append({'num_code': 682,
                       'en_short_name':'Saudi Arabia',
                       'category': 'Arabian'}, ignore_index=True).append({'num_code': 608,
                       'en_short_name':'Philippines',
                       'category': 'Filipino'}, ignore_index=True).append({'num_code': 348,
                       'en_short_name':'Hungary',
                       'category': 'Hungarian'}, ignore_index=True).append({'num_code': 408,
                       'en_short_name':"Korea (Democratic People's Republic of)",
                       'category': 'Korean'}, ignore_index=True).append({'num_code': 410,
                       'en_short_name':'Korea (Republic of)',
                       'category': 'Korean'}, ignore_index=True).append({'num_code': 418,
                       'en_short_name':"Lao People's Democratic Republic",
                       'category': 'Laotian'}, ignore_index=True).append({'num_code': 364,
                       'en_short_name':'Iran',
                       'category': 'Persian'}, ignore_index=True).append({'num_code': 703,
                       'en_short_name':'Slovakia',
                       'category': 'Slovakian'}, ignore_index=True).append({'num_code': 706,
                       'en_short_name':'Somalia',
                       'category': 'Somali'}, ignore_index=True).append({'num_code': 144,
                       'en_short_name':'Sri lanka',
                       'category': 'Sri lankan'}, ignore_index=True).append({'num_code': 706,
                       'en_short_name':'Somalia',
                       'category': 'Somali'}, ignore_index=True).append({'num_code': 158,
                       'en_short_name':'Taiwan, Province of China',
                       'category': 'Taiwanese'}, ignore_index=True).append({'num_code': 860,
                       'en_short_name':'Uzbekistan',
                       'category': 'Uzbek'}, ignore_index=True).append({'num_code': 316,
                       'en_short_name':'Guam',
                       'category': 'Guamanian'}, ignore_index=True).append({'num_code': 840,
                       'en_short_name':'United States of America',
                       'category': 'Tradamerican'}, ignore_index=True).append({'num_code': 840,
                       'en_short_name':'United States of America',
                       'category': 'Newamerican'}, ignore_index=True)

In [20]:
country_lookup.shape

(57, 3)

In [21]:
country_lookup['category'] = country_lookup['category'].str.lower()

In [22]:
country_lookup.head()

,num_code,en_short_name,category
0,32,Argentina,argentine
1,51,Armenia,armenian
2,36,Australia,australian
3,40,Austria,austrian
4,50,Bangladesh,bangladeshi


Remaining categories that cannot be mapped to a country

In [23]:
[x for x in cetegories['category'].to_list() if x not in country_lookup['category'].to_list()]

['Afghani',
 'African',
 'Arabian',
 'Argentine',
 'Armenian',
 'Australian',
 'Austrian',
 'Bangladeshi',
 'Basque',
 'Belgian',
 'Brazilian',
 'British',
 'Burmese',
 'Cajun',
 'Cambodian',
 'Caribbean',
 'Catalan',
 'Chinese',
 'Cuban',
 'Czech',
 'Eritrean',
 'Ethiopian',
 'Filipino',
 'French',
 'Georgian',
 'German',
 'Halal',
 'Hawaiian',
 'Himalayan',
 'Honduran',
 'Hungarian',
 'Iberian',
 'Indonesian',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Kosher',
 'Laotian',
 'Latin',
 'Malaysian',
 'Mediterranean',
 'Mexican',
 'Mideastern',
 'Modern_european',
 'Mongolian',
 'Moroccan',
 'Newamerican',
 'Newmexican',
 'Pakistani',
 'Panasian',
 'Persian',
 'Peruvian',
 'Piadina',
 'Polish',
 'Polynesian',
 'Portuguese',
 'Russian',
 'Scandinavian',
 'Scottish',
 'Singaporean',
 'Slovakian',
 'Somali',
 'Spanish',
 'Srilankan',
 'Syrian',
 'Taiwanese',
 'Thai',
 'Tradamerican',
 'Turkish',
 'Ukrainian',
 'Uzbek',
 'Vietnamese',
 'Nicaraguan',
 'Bulgarian',
 'Guamanian']

## Aggregate country name, country code, and human-friendly category name together

In [48]:
US_catagories = pd.read_csv('Yelp_US_categories.csv')

In [49]:
food_diversity_cleaned = pd.merge(food_diversity, US_catagories, how = 'left', left_on = 'category', right_on = 'alias')\
    .rename(columns = {'title':'category_name'})
food_diversity_cleaned.shape

(2672, 9)

In [50]:
food_diversity_cleaned.head(1)

,query,category,total_business_count,sample_rating,sample_review_count,Unnamed: 0,alias,category_name,parents
0,"New York, New York",afghani,18,3.75,150.111,1,afghani,Afghan,restaurants


In [51]:
country_lookup.drop_duplicates(inplace = True)

In [52]:
food_diversity_cleaned = pd.merge(food_diversity_cleaned, country_lookup, how = 'left', left_on = 'category', right_on = 'category')\
                        [['query','category','category_name', 'num_code', 'en_short_name', 'total_business_count', 'sample_rating', 'sample_review_count']]\
                        .rename(columns = {
                            'query': 'city',
                            'num_code':'M49_country_code',
                            'en_short_name': 'country'
                        })
food_diversity_cleaned.shape

(2723, 8)

In [53]:
food_diversity_cleaned.head()

,city,category,category_name,M49_country_code,country,total_business_count,sample_rating,sample_review_count
0,"New York, New York",afghani,Afghan,NaN,NaN,18,3.75,150.111
1,"New York, New York",african,African,NaN,NaN,48,4.11458,115.646
2,"New York, New York",arabian,Arabian,682.0,Saudi Arabia,5,3.9,51.6
3,"New York, New York",argentine,Argentine,32.0,Argentina,21,3.90476,182.857
4,"New York, New York",armenian,Armenian,51.0,Armenia,5,4.5,27.4


## Export clean, geo-marked data to CSV

In [55]:
food_diversity_cleaned.to_csv('food_diversity.csv')